In [ ]:
!pip install azure-search-documents==11.4.0b6
!pip install azure-identity
!pip install langchain==0.0.248
!pip install openai==0.27.8
!pip install python-dotenv==1.0.0
!pip install uvicorn==0.23.2
!pip install fastapi==0.100.1
!pip install tiktoken
!pip install pypdf

In [ ]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents import initialize_agent, Tool, AgentType

openai_api_key = "{OpenAIKEY}"
openai_env= "{OpenAIENVURL}"
openai_model = "text-embedding-ada-002"
azure_search_endpoint = "{CosmosDBURL}"
azure_search_key ="{CosmosDBKEY}"
load_dotenv(verbose=True)

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_key = openai_api_key
openai.api_base = openai_env
model: str = openai_model

In [ ]:
embeddings = OpenAIEmbeddings(deployment=openai_model, openai_api_key=openai_api_key, model=openai_model)
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, deployment=model, model=model, chunk_size=1, openai_api_base=openai_env, openai_api_type="azure" )
index_name: str = "lab-ai-001" #banco de dados em vetor
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=azure_search_endpoint,
    azure_search_key=azure_search_key,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

vector_store = AzureSearch(
    azure_search_endpoint= azure_search_endpoint,
    azure_search_key= azure_search_key,
    index_name="lab-ai-001",
    embedding_function=embeddings.embed_query,
)


In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
vector_store.add_documents(documents=docs);

In [ ]:
llm = ChatOpenAI(engine='gpt4', verbose=False,openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

In [ ]:
query = ""
result = qa({"query": query})
result["result"]